In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-2023/dataset/sample_submission.csv
/kaggle/input/amazon-2023/dataset/train.csv
/kaggle/input/amazon-2023/dataset/test.csv


In [3]:
# importing libraries 
import pandas as pd
import numpy as np
import scipy

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

import gc

In [4]:
#loading the dataset
print("Reading in Data")

df_train = pd.read_csv('/kaggle/input/amazon-2023/dataset/train.csv',nrows=220000)
df_test = pd.read_csv('/kaggle/input/amazon-2023/dataset/test.csv')

Reading in Data


In [5]:
#dataset information
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [7]:
df_test.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220000 entries, 0 to 219999
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   PRODUCT_ID       220000 non-null  int64  
 1   TITLE            219998 non-null  object 
 2   BULLET_POINTS    138079 non-null  object 
 3   DESCRIPTION      106665 non-null  object 
 4   PRODUCT_TYPE_ID  220000 non-null  int64  
 5   PRODUCT_LENGTH   220000 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 10.1+ MB


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734736 entries, 0 to 734735
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   PRODUCT_ID       734736 non-null  int64 
 1   TITLE            734731 non-null  object
 2   BULLET_POINTS    458814 non-null  object
 3   DESCRIPTION      354735 non-null  object
 4   PRODUCT_TYPE_ID  734736 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 28.0+ MB


In [10]:
#concating the datasets for better accuracy calculation
df = pd.concat([df_train, df_test], 0)
nrow_train = df_train.shape[0]
y_train = np.log1p(df_train["PRODUCT_LENGTH"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


In [11]:
#saving the ram usage, minimising it
del df_train
gc.collect()

23

In [12]:
print(df.memory_usage(deep = True))

Index                7637888
PRODUCT_ID           7637888
TITLE              140623179
BULLET_POINTS      447771916
DESCRIPTION        410236235
PRODUCT_TYPE_ID      7637888
PRODUCT_LENGTH       7637888
dtype: int64


In [13]:
df["DESCRIPTION"] = df["DESCRIPTION"].fillna("None")
df["TITLE"] = df["TITLE"].fillna("Unknown")
df["BULLET_POINTS"] = df["BULLET_POINTS"].fillna("Not Known")

In [14]:
# declaring variables
NAME_MIN_DF = 10
MAX_FEAT_DESCP = 50000


In [15]:
#encoding the text fields
print("Encodings")
count = CountVectorizer(min_df=NAME_MIN_DF)
X_name = count.fit_transform(df["TITLE"])

Encodings


In [16]:
# count vectoriser to convert to matrix of tokens
print("Bulletpoints Encoders")
unique_categories = pd.Series("/".join(df["BULLET_POINTS"].unique().astype("str")).split("/")).unique()
count_category = CountVectorizer()
X_bulletpoints = count_category.fit_transform(df["BULLET_POINTS"])

Bulletpoints Encoders


In [17]:
#tfidf for description text field
print("Descp encoders")
count_descp = TfidfVectorizer(max_features = MAX_FEAT_DESCP, 
                              ngram_range = (1,3),
                              stop_words = "english")
X_descp = count_descp.fit_transform(df["DESCRIPTION"]) 

Descp encoders


In [18]:
# dummy encoders for rest of the remaining fields
print("Dummy Encoders")
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(df[[
    "PRODUCT_ID", "PRODUCT_TYPE_ID"]], sparse = True).values)

X = scipy.sparse.hstack((X_dummies, 
                         X_descp,
                         X_bulletpoints,
                         X_name)).tocsr()

Dummy Encoders


In [19]:
#shape of fields
print([X_dummies.shape, X_bulletpoints.shape, 
       X_name.shape, X_descp.shape])

[(954736, 2), (954736, 228879), (954736, 46124), (954736, 50000)]


In [20]:
# using ridge regression 
X_train = X[:nrow_train]
model = Ridge(solver = "lsqr", fit_intercept=False)

In [21]:
#training the model
print("Fitting Model")
model.fit(X_train, y_train)

Fitting Model


Ridge(fit_intercept=False, solver='lsqr')

In [22]:
# making predictions
X_test = X[nrow_train:]
preds = model.predict(X_test)

In [24]:
#storing the results into a csv file format
df_test["PRODUCT_LENGTH"] = np.expm1(preds)
df_test[["PRODUCT_ID", "PRODUCT_LENGTH"]].to_csv("submission_ridge_test.csv", index = False)